In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('../input/annex1.csv')
df.head()

,Item Code,Item Name,Category Code,Category Name
0,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables
1,102900005115199,Sichuan Red Cedar,1011010101,Flower/Leaf Vegetables
2,102900005115625,Local Xiaomao Cabbage,1011010101,Flower/Leaf Vegetables
3,102900005115748,White Caitai,1011010101,Flower/Leaf Vegetables
4,102900005115762,Amaranth,1011010101,Flower/Leaf Vegetables


In [3]:
df1 = pd.read_csv('../input/annex2.csv')
df1.head()

,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (RMB/kg),Sale or Return,Discount (Yes/No)
0,2020-07-01,09:15:07.924,102900005117056,0.396,7.6,sale,No
1,2020-07-01,09:17:27.295,102900005115960,0.849,3.2,sale,No
2,2020-07-01,09:17:33.905,102900005117056,0.409,7.6,sale,No
3,2020-07-01,09:19:45.450,102900005115823,0.421,10.0,sale,No
4,2020-07-01,09:20:23.686,102900005115908,0.539,8.0,sale,No


In [4]:
# Join the both DataFrames

merge = pd.merge(df, df1, on ='Item Code')
merge.head()

,Item Code,Item Name,Category Code,Category Name,Date,Time,Quantity Sold (kilo),Unit Selling Price (RMB/kg),Sale or Return,Discount (Yes/No)
0,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables,2020-10-30,10:02:13.748,0.743,5.0,sale,No
1,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables,2020-10-30,10:48:42.707,0.502,5.0,sale,No
2,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables,2020-10-30,11:10:13.023,0.301,5.0,sale,No
3,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables,2020-10-30,12:34:32.084,0.382,5.0,sale,No
4,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables,2020-10-30,13:33:59.748,0.261,5.0,sale,No


In [5]:
df = merge
intended_df_size_in_MB = 256
factor = intended_df_size_in_MB*(2**20)/df.memory_usage(index=True).sum()
if int(factor) > 0:
    df = pd.concat([df]*int(factor), ignore_index=True)
else:
    rowCount = int(df.shape[0]*factor)
    df = df[0:rowCount]
merge = df
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2635509 entries, 0 to 2635508
Data columns (total 10 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Item Code                    int64  
 1   Item Name                    object 
 2   Category Code                int64  
 3   Category Name                object 
 4   Date                         object 
 5   Time                         object 
 6   Quantity Sold (kilo)         float64
 7   Unit Selling Price (RMB/kg)  float64
 8   Sale or Return               object 
 9   Discount (Yes/No)            object 
dtypes: float64(2), int64(2), object(6)
memory usage: 201.1+ MB


# Data Preparation

In [6]:
# Rename Columns
merge = merge.rename(columns={'Quantity Sold (kilo)':'Quantity Sold', 'Unit Selling Price (RMB/kg)':'Unit Selling Price','Discount (Yes/No)':'Discount'})

In [7]:
#Change datatype of date column.

merge['Date'] = pd.to_datetime(merge['Date'])

In [8]:
merge.isnull().sum()

Item Code             0
Item Name             0
Category Code         0
Category Name         0
Date                  0
Time                  0
Quantity Sold         0
Unit Selling Price    0
Sale or Return        0
Discount              0
dtype: int64

# Exploratory Data Analysis

In [9]:
Category_count = merge['Category Name'].value_counts()
Category_count

Flower/Leaf Vegetables         995904
Capsicum                       623988
Edible Mushroom                445272
Cabbage                        259710
Aquatic Tuberous Vegetables    175941
Solanum                        134694
Name: Category Name, dtype: int64

In [10]:
top_10_products = merge['Item Name'].value_counts()[:10]
top_10_products

Wuhu Green Pepper (1)    209835
Broccoli                 176718
Xixia Mushroom (1)       142527
Yunnan Shengcai          119661
Net Lotus Root (1)       117855
Eggplant (2)              82365
Paopaojiao (Jingpin)      76014
Yunnan Lettuces           69795
Luosi Pepper              68034
Chinese Cabbage           57444
Name: Item Name, dtype: int64

In [11]:
# Creating sales column

merge['Sales'] = merge['Unit Selling Price'] * merge['Quantity Sold']

In [12]:
Item_sales = merge.groupby('Item Name',as_index=False)['Sales'].sum().sort_values(by='Sales',ascending=False).head(10)
Item_sales

,Item Name,Sales
24,Broccoli,809642.8716
136,Net Lotus Root (1),634956.3039
223,Xixia Mushroom (1),633595.1592
209,Wuhu Green Pepper (1),615341.0973
233,Yunnan Shengcai,389270.8845
45,Eggplant (2),353188.2717
142,Paopaojiao (Jingpin),286705.9017
114,Luosi Pepper,246028.0353
232,Yunnan Lettuces,211995.3183
32,Chinese Cabbage,197026.3098


In [13]:
#Add year column in dataframe
merge['Year'] = merge['Date'].dt.year

In [14]:
Year_wise_sales=merge.groupby('Year')['Sales'].sum()
Year_wise_sales

Year
2020    2.008588e+06
2021    3.301088e+06
2022    3.110317e+06
2023    1.689306e+06
Name: Sales, dtype: float64

In [15]:
Sale_Or_Return = merge['Sale or Return'].value_counts()
Sale_Or_Return = pd.DataFrame(Sale_Or_Return)
Sale_Or_Return

,Sale or Return
sale,2634126
return,1383


In [16]:
Discount = merge['Discount'].value_counts()
Discount = pd.DataFrame(Discount)
Discount

,Discount
No,2493411
Yes,142098
